In [ ]:
!pip install transformers==4.2.2



In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("nikokons/gpt2-greek")

train_path = '/content/iliadaodysseia.txt'



In [ ]:
from transformers import TextDataset,DataCollatorForLanguageModeling

def load_dataset(train_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)



    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,data_collator

train_dataset,data_collator = load_dataset(train_path,tokenizer)

In [ ]:
from transformers import Trainer, TrainingArguments, AutoModelWithLMHead

model = AutoModelWithLMHead.from_pretrained("nikokons/gpt2-greek")

training_args = TrainingArguments(
    output_dir="./gpt2-homer", #The output directory
    overwrite_output_dir=True, #overwrite the content of the output directory
    num_train_epochs=3, # number of training epochs
    per_device_train_batch_size=32, # batch size for training
    per_device_eval_batch_size=64,  # batch size for evaluation
    eval_steps = 400, # Number of update steps between two evaluations.
    save_steps=800, # after # steps model is saved
    warmup_steps=500,# number of warmup steps for learning rate scheduler
    prediction_loss_only=True,
    )

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,

   
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

In [ ]:
from transformers import pipeline

generator = pipeline('text-generation',model='./gpt2-homer', tokenizer='nikokons/gpt2-greek')



In [ ]:
result = generator('Και τότε ο Οδυσσέας ',
                   do_sample=True, 
    max_length=200, 
    top_p=0.92, 
    top_k=0
                    )[0]['generated_text']
# print(result)

In [ ]:
print(result)

Output generated:

Και τότε ο Οδυσσέας  του απηλογήθη κι είπε:
«Και τώρα, πηγαίνοντας στους θεούς τ' αρνιά τρανή σου πια βουλή δεν 'ρθε,
το ρίξει ο άλλος, ο Ευρύλοχος, ο Μενέλαος µε το σπήλιο,
γιατί απ' τις πιοτσιάδες απαράλλαχτες του είναι πια τόσο πως δεν σου διώχεις.
Κι αυτοί τα χωράφια και τα νερά των σκλάβων ξεκάνουν
και σκόρπισαν στο Παρπησσό στη γη τον πόλεµο, κι ο Μενέλαος τα φαγιά και το ξάρπιο
σαν τα σπαθιά τους' το ψηλό κρασί χτυπιούνται και τα κροσσωµένη ψυχανθές,
και τα πολλά σε κάθιζαν στο ανάµερο οχτρεύει;

Πο γιεσός τους πολλούς δίνει' κι


The output can be improved, while it is syntactically not bad , some of the word sequences are not meaningful
